In [1]:
!pip install yfinance #API для выгрузки рыночной информации

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Operation cancelled by user


In [ ]:
#устанавливаем необходимые для анализа пакеты
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import requests as requests
from google.colab import files

In [ ]:
 #вспомогательный объект Series для отбора данных на необходимую дату
date = ['2019-03-01','2019-06-01','2019-09-01','2019-12-01','2020-03-01',
        '2020-06-01','2020-09-01','2020-12-01','2021-03-01','2021-06-01',
        '2021-09-01','2021-12-01']

In [ ]:
#ссылка на персональный Google disk для доступа к файлу с отобранными тикерами 
url = "https://docs.google.com/spreadsheets/d/1uwaBuXf_x8SSk-fsWAheyB_OaN99bNuc/edit?usp=share_link&ouid=110986586806883017870&rtpof=true&sd=true"
path = 'https://drive.google.com/uc?id='+url.split('/')[-2]
tickers = pd.read_excel(path)

In [ ]:
tickers.head(5) ##содержание таблицы

In [ ]:
tck = tickers['Symbol'].squeeze() #извлекаем из матрицы тикеры в виде списка
tck

In [ ]:
#с помощью установленного расширения извлекаем информацию о предприятиях
#и рыночную информацию по кварталам
tickers_data = {} 
stock_data = {}

for ticker in tck:
  ticker_object = yf.Ticker(ticker)
  ticker_market = yf.download(ticker, 
                              start = "2019-01-31", 
                              end = "2021-12-31", interval = "1mo")

  temp = pd.DataFrame.from_dict(ticker_object.info, orient="index")
  temp.reset_index(inplace=True)
  temp.columns = ["Attribute", "Recent"]

  tickers_data[ticker] = temp
  stock_data[ticker] = ticker_market

In [ ]:
#полученную информацию собираем в единые удобные для анализа датасеты
combined_data = pd.concat(tickers_data)
combined_stock_data = pd.concat(stock_data)

combined_data = combined_data.reset_index()
combined_stock_data = combined_stock_data.reset_index()

In [ ]:
#удаляем ненужный столбец в датасете с общей информацией
del combined_data["level_1"] 
#отбираем необходимые данные по торгам
query = combined_stock_data.Date.isin(['2019-03-01','2019-06-01','2019-09-01',
                                       '2019-12-01','2020-03-01','2020-06-01',
                                       '2020-09-01','2020-12-01','2021-03-01',
                                       '2021-06-01','2021-09-01','2021-12-01']) 
combined_stock_data = combined_stock_data[query]

In [ ]:
combined_data

In [ ]:
combined_stock_data

In [ ]:
#получаем таблицу, в которой для каждого тикера приводится отрасль (industry)
industry = combined_data[combined_data["Attribute"]=="industry"].reset_index()
del industry["index"] #удаляем ненужный столбец
industry

In [ ]:
#выгружаем оба файла для составления датасета
industry.to_excel('industry.xlsx')
combined_stock_data.to_csv('stock_full.csv')

files.download('industry.xlsx')
files.download('stock_full.csv')

In [ ]:
#аналогичным образом извлекаем квартальные данные c Macrotrends
#понадобятся данные по EPS, общему объему активов, НМА, количеству акций,
#FCF и Book value на одну акцию, затраты на R&D, показатель ROA 
eps_data = {}
assets_data = {}
intangible_data = {}
shares_data = {}
fcf_data = {}
bv_data = {}
RD_data = {}
revenue_data = {}
ROA_data = {}

In [ ]:
for ticker in tck:
  url_eps = 'https://www.macrotrends.net/stocks/charts/' + ticker + '/caterpillar/eps-earnings-per-share-diluted'
  eps_histo = pd.read_html(url_eps)[1]
  eps_histo = eps_histo.set_axis(['Date',  'Quartely'], axis = 1)
  eps_data[ticker] = eps_histo

  url_assets = 'https://www.macrotrends.net/stocks/charts/' + ticker + '/caterpillar/total-assets'
  asset_histo = pd.read_html(url_assets)[1]
  asset_histo = asset_histo.set_axis(['Date',  'Quartely'], axis = 1) 
  assets_data[ticker] = asset_histo

  url_intang = 'https://www.macrotrends.net/stocks/charts/' + ticker + '/caterpillar/goodwill-intangible-assets-total'
  intangible_histo = pd.read_html(url_intang)[1]
  intangible_histo = intangible_histo.set_axis(['Date',  'Quartely'], axis = 1)
  intangible_data[ticker] = intangible_histo 

  url_shares = 'https://www.macrotrends.net/stocks/charts/' + ticker + '/caterpillar/shares-outstanding'
  shares_histo = pd.read_html(url_shares)[1]
  shares_histo = shares_histo.set_axis(['Date',  'Quartely'], axis = 1)
  shares_data[ticker] = shares_histo

  url_fcf = 'https://www.macrotrends.net/stocks/charts/' + ticker + '/caterpillar/price-fcf'
  fcf_histo = pd.read_html(url_fcf)[0]
  fcf_histo = fcf_histo.set_axis(['Date',  'Price', 'TTM FCF', 'P/FCF'], axis = 1)
  fcf_histo = fcf_histo[['Date', 'TTM FCF']]
  fcf_data[ticker] = fcf_histo

  url_bv = 'https://www.macrotrends.net/stocks/charts/' + ticker + '/caterpillar/price-book'
  bv_histo = pd.read_html(url_bv)[0]
  bv_histo = bv_histo.set_axis(['Date',  'Price', 'Book Value', 'P/BV'], axis = 1)
  bv_histo = bv_histo[['Date','Book Value']]
  bv_data[ticker] = bv_histo  

  url_RD = 'https://www.macrotrends.net/stocks/charts/' + ticker + '/caterpillar/research-development-expenses'
  RD_histo = pd.read_html(url_RD)[1]
  RD_histo = RD_histo.set_axis(['Date',  'Quartely'], axis = 1)
  RD_data[ticker] = RD_histo 

  url_rev = 'https://www.macrotrends.net/stocks/charts/' + ticker + '/caterpillar/revenue'
  rev_histo = pd.read_html(url_rev)[1]
  rev_histo = rev_histo.set_axis(['Date',  'Quartely'], axis = 1)
  revenue_data[ticker] = rev_histo

  url_roa = 'https://www.macrotrends.net/stocks/charts/' + ticker + '/caterpillar/roa'
  roa_histo = pd.read_html(url_roa)[0]
  roa_histo = roa_histo.set_axis(['Date',  'TTM Net Income', 'Total assets', 'Quartely'], axis = 1)
  ROA_data[ticker] = roa_histo

In [ ]:
#полученную информацию собираем в удобные для анализа датасеты
combined_eps_data = pd.concat(eps_data)
combined_asset_data = pd.concat(assets_data)
combined_intangible_data = pd.concat(intangible_data)
combined_shares_data = pd.concat(shares_data)
combined_fcf_data = pd.concat(fcf_data)
combined_bv_data = pd.concat(bv_data)
combined_RD_data = pd.concat(RD_data)
combined_rev_data = pd.concat (revenue_data)
combined_roa_data = pd.concat (ROA_data)

combined_eps_data = combined_eps_data.reset_index()
combined_asset_data = combined_asset_data.reset_index()
combined_intangible_data = combined_intangible_data.reset_index()
combined_shares_data = combined_shares_data.reset_index()
combined_fcf_data = combined_fcf_data.reset_index()
combined_bv_data = combined_bv_data.reset_index()
combined_RD_data = combined_RD_data.reset_index()
combined_rev_data = combined_rev_data.reset_index()
combined_roa_data = combined_roa_data.reset_index()

In [ ]:
#удаляем ненужный столбец
del combined_eps_data['level_1']
del combined_asset_data['level_1']
del combined_intangible_data['level_1']
del combined_shares_data['level_1']
del combined_fcf_data['level_1']
del combined_bv_data['level_1']
del combined_RD_data['level_1']
del combined_rev_data['level_1']
del combined_roa_data['level_1']

In [ ]:
combined_eps_data

In [ ]:
combined_asset_data

In [ ]:
combined_intangible_data

In [ ]:
combined_shares_data

In [ ]:
combined_fcf_data

In [ ]:
combined_bv_data

In [ ]:
combined_RD_data

In [ ]:
combined_rev_data

In [ ]:
combined_roa_data

In [ ]:
#формируем файл для расчета TTM EPS
pe_data = combined_eps_data[(combined_eps_data.Date <= '2021-12-31') & (combined_eps_data.Date >= '2018-01-31')]

#формируем файлы для расчета доли нематериальных активов
asset_data = combined_asset_data[(combined_asset_data.Date <= '2021-12-31') & (combined_asset_data.Date >= '2019-01-31')]
intangible_data = combined_intangible_data[(combined_intangible_data.Date <= '2021-12-31') & (combined_intangible_data.Date >= '2019-01-31')]

#формируем файл для вычисления количества акций
shares_data = combined_shares_data[(combined_shares_data.Date <= '2021-12-31') & (combined_shares_data.Date >= '2019-01-31')]

#формируем файл по FCF на акцию
fcf_data = combined_fcf_data[(combined_fcf_data.Date <= '2021-12-31') & (combined_fcf_data.Date >= '2019-01-31')]

#формируем файл по Book value на акцию
bv_data = combined_bv_data[(combined_bv_data.Date <= '2021-12-31') & (combined_bv_data.Date >= '2019-01-31')]

#формируем файл для вычисления доли инвестиций в R&D
RD_data = combined_RD_data[(combined_RD_data.Date <= '2021-12-31') & (combined_RD_data.Date >= '2019-01-31')]
rev_data = combined_rev_data[(combined_rev_data.Date <= '2021-12-31') & (combined_rev_data.Date >= '2019-01-31')]

#формируем файл по ROA
roa_data = combined_roa_data[(combined_roa_data.Date <= '2021-12-31') & (combined_roa_data.Date >= '2019-01-31')]

In [ ]:
#скачиваем файл по EPS для датасета
pe_data.to_excel('eps_data.xlsx')
files.download('eps_data.xlsx')

In [ ]:
#скачиваем файл по активам для датасета
asset_data.to_excel('asset_data.xlsx')
files.download('asset_data.xlsx')

In [ ]:
#скачиваем файл по НМА для датасета
intangible_data.to_excel('intangible_data.xlsx')
files.download('intangible_data.xlsx')

In [ ]:
#скачиваем файл по количеству акций для датасета
shares_data.to_excel('shares_data.xlsx')
files.download('shares_data.xlsx')

In [ ]:
#скачиваем файл по FCF на акцию для датасета
fcf_data.to_excel('fcf_data.xlsx')
files.download('fcf_data.xlsx')

In [ ]:
#скачиваем файл по Book value на акцию для датасета
bv_data.to_excel('bv_data.xlsx')
files.download('bv_data.xlsx')

In [ ]:
#скачиваем файл по расходам на R&D для датасета
RD_data.to_excel('RD_data.xlsx')
files.download('RD_data.xlsx')

In [ ]:
#скачиваем файл по выручке для датасета
rev_data.to_excel('rev_data.xlsx')
files.download('rev_data.xlsx')

In [ ]:
#скачиваем файл по ROA для датасета
roa_data.to_excel('roa_data.xlsx')
files.download('roa_data.xlsx')

На основе файлов, полученных с помощью веб скрапинга данных с Macrotrends и Yahoo Finance, был собран датасет, который будет использован для оценки

In [ ]:

dataset_url  = 'https://docs.google.com/spreadsheets/d/1gvwNb4oduAEL5RfU9Eei6OWDGHkRVY3M/edit?usp=share_link&ouid=110986586806883017870&rtpof=true&sd=true'
dataset_path = 'https://drive.google.com/uc?id=' + dataset_url.split('/')[-2]
dataset = pd.read_excel(dataset_path)

In [ ]:
dataset.head()

In [ ]:
dataset.isnull().sum(axis=0) #смотрим на количество пропущенных значений

In [ ]:
#удаляем строки, в которых есть хоть одно NaN
dataset = dataset.dropna() 
#удаляем ненужные столбцы
dataset.drop(dataset.columns[[3,4,5,6,8,9,10,11,12]], axis=1, inplace = True)

In [ ]:
#переименовываем столбец
dataset.rename({'Intangible assets to assets ratio' : 'Intangible assets, share'}, axis=1, inplace = True)

In [ ]:
#выводим общие показатели в отдельную таблицу
dataset.describe().to_excel('description.xlsx')

In [ ]:
#скачиваем
files.download('description.xlsx')

In [ ]:
#создаем отдельные датасеты по каждой индустрии для графика
farm_heavy = dataset[dataset.Industry == 'Farm & Heavy Construction Machinery']
spec_ind_mach = dataset[dataset.Industry == 'Specialty Industrial Machinery']
elec_equip = dataset[dataset.Industry == 'Electrical Equipment & Parts']
metal_fabr = dataset[dataset.Industry == 'Metal Fabrication']
eng_contr = dataset[dataset.Industry == 'Engineering & Construction']

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(13, 7) # Задаём размеры графика.

# Подписываем и отрисовываем точки.
ax.scatter(farm_heavy['Date'], farm_heavy['Capitalization, mln'], label="Farm & Heavy Construction Machinery", facecolor="blue", alpha=0.5)
ax.scatter(spec_ind_mach['Date'], spec_ind_mach['Capitalization, mln'], label="Specialty Industrial Machinery", facecolor="green", alpha=0.5)
ax.scatter(elec_equip['Date'], elec_equip['Capitalization, mln'], label="Electrical Equipment & Parts", facecolor="red", alpha=0.5)
ax.scatter(metal_fabr['Date'], metal_fabr['Capitalization, mln'], label="Metal Fabrication", facecolor="violet", alpha=0.5)
ax.scatter(eng_contr['Date'], eng_contr['Capitalization, mln'], label="Engineering & Construction", facecolor="yellow", alpha=0.5)

ax.set_xlabel("Дата")
ax.set_ylabel("Капитализация, млн долларов")
ax.grid()
ax.set_title("Промышленные предприятия")
ax.legend()

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [ ]:
#заменяем текстовые значения индустрии на числовую кодировку
new = le.fit_transform(dataset['Industry'])
dataset['Industry'] = new

In [ ]:
dataset.head()

In [ ]:
#строим корреляционную матрицу
sub_ds = dataset.iloc[:, 3:]
corr = sub_ds.corr()
corr.style.background_gradient(cmap='coolwarm')

In [ ]:

from statsmodels.stats.outliers_influence import variance_inflation_factor
  
# регрессоры
X = sub_ds.iloc[:,:-1]
  
# датафрейм для расчета VIF 
vif_data = pd.DataFrame()
vif_data["feature"] = X.columns
  
# расчет VIF
vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                          for i in range(len(X.columns))]
  
print(vif_data)


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

In [ ]:
#формируем датасеты по признаками и метке
X = dataset.iloc[:, 3:9]
y = dataset.iloc[:, 9:]

In [ ]:
X

In [ ]:
y

In [ ]:
#разделение на обучающую и тестировочную подвыборки
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=3)

In [ ]:
#формирование исходной модели с базовыми параметрами
model0 =  RandomForestRegressor(random_state=3) 

In [ ]:
#обучение модели
model0.fit(X_train, y_train)

In [ ]:
#предсказание
y_pred0 = model0.predict(X_test)

In [ ]:
# Mean absolute error (MAE)
mae = mean_absolute_error(y_test, y_pred0)

# Mean squared error (MSE)
mse = mean_squared_error(y_test, y_pred0)

# R-squared scores
r2 = r2_score(y_test, y_pred0)

# выводим метрики
print('Mean Absolute Error:', round(mae, 2))
print('Mean Squared Error:', round(mse, 2))
print('R-squared scores:', round(r2, 2))

In [ ]:
#строим точечный график для сравнения
x_axis = X_test[['Intangible assets, share']]

plt.scatter(x_axis, y_test, c = 'b', alpha = 0.5, marker = '.', label = 'фактические')
plt.scatter(x_axis, y_pred0, c = 'r', alpha = 0.5, marker = '.', label = 'предсказанные')
plt.xlabel('Нематериальные активы, доля')
plt.ylabel('Капитализация, млн долларов')
plt.grid(color = '#D3D3D3', linestyle = 'solid')
plt.legend(loc = 'upper right')
plt.xlim(0,1)
plt.show()

In [ ]:
#извлекаем данные по важности параметров
characteristics = X.columns
importances = list(model0.feature_importances_)
characteristics_importances = [(characteristic, round(importance, 2)) for characteristic, importance in zip(characteristics, importances)]
characteristics_importances = sorted(characteristics_importances, key = lambda x: x[1], reverse = True)
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in characteristics_importances]

In [ ]:
#строим столбчатую диаграмму для важности параметров 
plt.bar(characteristics, importances, orientation = 'vertical')
plt.xticks(rotation = 'vertical')
plt.ylabel('Среднее снижение загрязнения')
plt.xlabel('Признак')
plt.grid(axis = 'y', color = '#D3D3D3', linestyle = 'solid')
plt.show()

In [ ]:
#строим новую модель с увеличенным количеством деревьев
model =  RandomForestRegressor(n_estimators=500,                              
                               oob_score=True,
                               random_state=3, bootstrap = True) 

In [ ]:
#обучение модели
model.fit(X_train, y_train)

In [ ]:
#предсказание
y_pred = model.predict(X_test)

In [ ]:
# Mean absolute error (MAE)
mae = mean_absolute_error(y_test, y_pred)

# Mean squared error (MSE)
mse = mean_squared_error(y_test, y_pred)

# R-squared scores
r2 = r2_score(y_test, y_pred)

# выводим метрики
print('Mean Absolute Error:', round(mae, 2))
print('Mean Squared Error:', round(mse, 2))
print('R-squared scores:', round(r2, 2))

In [ ]:
#строим точечный график для сравнения
x_axis = X_test[['Intangible assets, share']]

plt.scatter(x_axis, y_test, c = 'b', alpha = 0.5, marker = '.', label = 'фактические')
plt.scatter(x_axis, y_pred, c = 'r', alpha = 0.5, marker = '.', label = 'предсказанные')
plt.xlabel('Нематериальные активы, доля')
plt.ylabel('Капитализация, млн долларов')
plt.grid(color = '#D3D3D3', linestyle = 'solid')
plt.legend(loc = 'upper right')
plt.xlim(0,1)
plt.show()

In [ ]:
#извлекаем данные по важности параметров
characteristics = X.columns
importances = list(model.feature_importances_)
characteristics_importances = [(characteristic, round(importance, 2)) for characteristic, importance in zip(characteristics, importances)]
characteristics_importances = sorted(characteristics_importances, key = lambda x: x[1], reverse = True)
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in characteristics_importances]

In [ ]:
#строим столбчатую диаграмму для важности параметров 
plt.bar(characteristics, importances, orientation = 'vertical')
plt.xticks(rotation = 'vertical')
plt.ylabel('Важность')
plt.xlabel('Признак')
plt.grid(axis = 'y', color = '#D3D3D3', linestyle = 'solid')
plt.show()

In [ ]:
#формируем новый усеченный датасет
X1 = dataset[['ROA','P/BV', 'Intangible assets, share', 'R/D to Revenue ratio']]
y1 = dataset.iloc[:, 9:]

In [ ]:
#разделение на обучающую и тестировочную подвыборки
X1_train, X1_test, y1_train, y1_test = train_test_split(X1,y1,test_size=0.2,random_state=3)

In [ ]:
#формируем исходную модель
model1 =  RandomForestRegressor(n_estimators=100,                              
                               oob_score=True,
                               random_state=3, bootstrap = True) 

In [ ]:
#обучение модели
model1.fit(X1_train, y1_train)

In [ ]:
#предсказание
y1_pred = model1.predict(X1_test)

In [ ]:
# Mean absolute error (MAE)
mae = mean_absolute_error(y1_test, y1_pred)

# Mean squared error (MSE)
mse = mean_squared_error(y1_test, y1_pred)

# R-squared scores
r2 = r2_score(y1_test, y1_pred)

# выводим метрики
print('Mean Absolute Error:', round(mae, 2))
print('Mean Squared Error:', round(mse, 2))
print('R-squared scores:', round(r2, 2))

In [ ]:
#строим точечный график для сравнения
x_axis = X1_test[['Intangible assets, share']]

plt.scatter(x_axis, y1_test, c = 'b', alpha = 0.5, marker = '.', label = 'фактические')
plt.scatter(x_axis, y1_pred, c = 'r', alpha = 0.5, marker = '.', label = 'предсказанные')
plt.xlabel('Нематериальные активы, доля')
plt.ylabel('Капитализация, млн долларов')
plt.grid(color = '#D3D3D3', linestyle = 'solid')
plt.legend(loc = 'upper right')
plt.xlim(0,1)
plt.show()

In [ ]:
#задаем параметры для количества деревьев
estimators = [50, 60, 70, 80, 90, 100, 120, 130, 140, 150, 160, 170, 180, 190, 200]

In [ ]:
#пустой словарь
estim = {}

In [ ]:
#перебор параметров с выгрузкой показателей
for n in estimators:
  model =  RandomForestRegressor(n_estimators=n, oob_score=True, random_state=3, bootstrap = True)
  model.fit(X1_train, y1_train)
  y_pred = model.predict(X1_test)
  MAE = mean_absolute_error(y1_test, y_pred)
  MSE = mean_squared_error(y1_test, y_pred)
  R2 = r2_score(y1_test, y_pred)
  concat = [MAE, MSE, R2]
  estim[n] = concat

In [ ]:
#формируем матрицу, переименовываем столбцы
estim1 = pd.DataFrame.from_dict(estim, orient='index').reset_index()
estim1.rename(columns={'index':'estimators',0:'MAE',1:'MSE',2:'R2'}, inplace=True)

In [ ]:
estim1

In [ ]:
#строим график изменения R2
plt.plot(estim1[['estimators']],estim1[['R2']])
plt.xlabel('Количество деревьев')
plt.ylabel('Значение R2')
plt.grid(color = '#D3D3D3', linestyle = 'solid')

In [ ]:
#формируем новую модель
model2 =  RandomForestRegressor(n_estimators=120,                              
                               oob_score=True,
                               random_state=3, bootstrap = True) 

In [ ]:
#обучение модели
model2.fit(X1_train, y1_train)

In [ ]:
#предсказание
y2_pred = model2.predict(X1_test)

In [ ]:
#оцениваем среднее количество узлов и среднюю максимальную глубину дерева
n_nodes = []
max_depths = []

for ind_tree in model2.estimators_:
    n_nodes.append(ind_tree.tree_.node_count)
    max_depths.append(ind_tree.tree_.max_depth)
    
print(f'Average number of nodes {int(np.mean(n_nodes))}')
print(f'Average maximum depth {int(np.mean(max_depths))}')

In [ ]:
#задаем параметры для глубины деревьев
depth = [12, 14, 16, 18, 20, 22, 24, 26, 28]

In [ ]:
#пустой словарь
dep = {}

In [ ]:
#перебор параметров с выгрузкой показателей
for n in depth:
  model =  RandomForestRegressor(n_estimators=120, max_depth=n, oob_score=True, random_state=3, bootstrap = True)
  model.fit(X1_train, y1_train)
  y_pred = model.predict(X1_test)
  MAE = mean_absolute_error(y1_test, y_pred)
  MSE = mean_squared_error(y1_test, y_pred)
  R2 = r2_score(y1_test, y_pred)
  concat = [MAE, MSE, R2]
  dep[n] = concat

In [ ]:
dep

In [ ]:
#формируем матрицу, переименовываем столбцы
dep1 = pd.DataFrame.from_dict(dep, orient='index').reset_index()
dep1.rename(columns={'index':'tree depth',0:'MAE',1:'MSE',2:'R2'}, inplace=True)

In [ ]:
#строим график изменения R2
plt.plot(dep1[['tree depth']],dep1[['R2']])
plt.xlabel('максимальная глубина деревьев')
plt.ylabel('Значение R2')
plt.grid(color = '#D3D3D3', linestyle = 'solid')

In [ ]:
#задаем параметры для механизма выбора количества признаков
features = ['sqrt', 'log2', 'auto', 1, 2, 3]

In [ ]:
#пустой словарь
feat = {}

In [ ]:
#перебор параметров с выгрузкой показателей
for n in features:
  model =  RandomForestRegressor(n_estimators=120, max_depth=14, max_features = n, oob_score=True, random_state=3, bootstrap = True)
  model.fit(X1_train, y1_train)
  y_pred = model.predict(X1_test)
  MAE = mean_absolute_error(y1_test, y_pred)
  MSE = mean_squared_error(y1_test, y_pred)
  R2 = r2_score(y1_test, y_pred)
  concat = [MAE, MSE, R2]
  feat[n] = concat

In [ ]:
#формируем матрицу, переименовываем столбцы
feat1 = pd.DataFrame.from_dict(feat, orient='index').reset_index()
feat1.rename(columns={'index':'Выбор признака',0:'MAE',1:'MSE',2:'R2'}, inplace=True)

In [ ]:
#вывод R2
feat1[['Выбор признака', 'R2']]

In [ ]:
#вывод MSE
feat1[['MSE']].values

In [ ]:
#формируем финальную модель
model3 =  RandomForestRegressor(n_estimators=120, max_depth=14, max_features='auto',                              
                               oob_score=True,
                               random_state=3, bootstrap = True) 

In [ ]:
#обучение модели
model3.fit(X1_train, y1_train)

In [ ]:
#предсказание
y3_pred = model3.predict(X1_test)

In [ ]:
# Mean absolute error (MAE)
mae = mean_absolute_error(y1_test, y3_pred)

# Mean squared error (MSE)
mse = mean_squared_error(y1_test, y3_pred)

# R-squared scores
r2 = r2_score(y1_test, y3_pred)

# выводим метрики
print('Mean Absolute Error:', round(mae, 2))
print('Mean Squared Error:', round(mse, 2))
print('R-squared scores:', round(r2, 3))

In [ ]:
#строим точечный график для сравнения
x_axis = X1_test[['Intangible assets, %']]

plt.scatter(x_axis, y1_test, c = 'b', alpha = 0.5, marker = '.', label = 'фактические')
plt.scatter(x_axis, y3_pred, c = 'r', alpha = 0.5, marker = '.', label = 'предсказанные')
plt.xlabel('Нематериальные активы, доля')
plt.ylabel('Капитализация, млн долларов')
plt.grid(color = '#D3D3D3', linestyle = 'solid')
plt.legend(loc = 'upper right')
plt.xlim(0,1)
plt.show()

In [ ]:
#извлекаем данные по важности параметров
characteristics = X1.columns
importances = list(model3.feature_importances_)
characteristics_importances = [(characteristic, round(importance, 2)) for characteristic, importance in zip(characteristics, importances)]
characteristics_importances = sorted(characteristics_importances, key = lambda x: x[1], reverse = True)
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in characteristics_importances]

In [ ]:
#строим столбчатую диаграмму для важности параметров 
plt.bar(characteristics, importances, orientation = 'vertical')
plt.xticks(rotation = 'vertical')
plt.ylabel('Важность')
plt.xlabel('Признак')
plt.grid(axis = 'y', color = '#D3D3D3', linestyle = 'solid')
plt.show()

In [ ]:
#загружаем датасет по российским компаниям
dataset_url  = 'https://docs.google.com/spreadsheets/d/1giXcVZZBKtNn17-pJoi1dwXEcBPL0sdl/edit?usp=share_link&ouid=110986586806883017870&rtpof=true&sd=true'
dataset_path = 'https://drive.google.com/uc?id=' + dataset_url.split('/')[-2]
dataset_rus = pd.read_excel(dataset_path)

In [ ]:
dataset_rus.head()

In [ ]:
dataset_rus.isnull().sum(axis=0) #смотрим на количество пропущенных значений

In [ ]:
#удаляем строки, в которых есть хоть одно NaN
dataset_rus = dataset_rus.dropna()

In [ ]:
#переименовываем столбец
dataset_rus.rename({'Intangible assets to assets ratio' : 'Intangible assets, share'}, axis=1, inplace = True)

In [ ]:
#выводим общие показатели в отдельную таблицу
dataset_rus.describe().to_excel('description.xlsx')

In [ ]:
files.download('description.xlsx')

In [ ]:
#строим корреляционную матрицу
corr = dataset_rus.corr()
corr.style.background_gradient(cmap='coolwarm')

In [ ]:

from statsmodels.stats.outliers_influence import variance_inflation_factor
  
# регрессоры
X = dataset_rus.iloc[:,2:-1]
X 
# датафрейм для расчета VIF 
vif_data = pd.DataFrame()
vif_data["feature"] = X.columns
  
# расчет VIF
vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                         for i in range(len(X.columns))]
  
print(vif_data)


In [ ]:
#формируем датасеты по признаками и метке
X_rus = dataset_rus.iloc[:, 2:-1]
y_rus = dataset_rus.iloc[:, 6:]

In [ ]:
y_rus

In [ ]:
#предсказание
y_rus_pred = model3.predict(X_rus)

In [ ]:
# Mean absolute error (MAE)
mae = mean_absolute_error(y_rus, y_rus_pred)

# Mean squared error (MSE)
mse = mean_squared_error(y_rus, y_rus_pred)

# R-squared scores
r2 = r2_score(y_rus, y_rus_pred)

# выводим метрики
print('Mean Absolute Error:', round(mae, 2))
print('Mean Squared Error:', round(mse, 2))
print('R-squared scores:', round(r2, 2))

In [ ]:
#строим точечный график для сравнения
x_axis = X_rus[['Intangible assets, share']]

plt.scatter(x_axis, y_rus, c = 'b', alpha = 0.5, marker = '.', label = 'фактические')
plt.scatter(x_axis, y_rus_pred, c = 'r', alpha = 0.5, marker = '.', label = 'предсказанные')
plt.xlabel('Нематериальные активы, доля')
plt.ylabel('Капитализация, млн долларов')
plt.grid(color = '#D3D3D3', linestyle = 'solid')
plt.legend(loc = 'upper right')
plt.xlim(0,0.125)
plt.ylim(0,20000)
plt.show()